In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images = train_images/255.0
test_iamges = test_images/255.0

In [6]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(filters = hp.Int('Conv_1_kernel',min_value = 64, max_value = 128, step = 16),
                        kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
                        activation = 'relu',
                       input_shape = (28,28,1)
        ),
        keras.layers.Conv2D(filters = hp.Int('Conv_2_kernel',min_value = 64, max_value = 128, step = 16),
                           kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
                           activation = 'relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(units = hp.Int('dense_1_units', min_value = 32, max_value = 132, step = 16),
                          activation = 'relu'
                          ),
        keras.layers.Dense(10, activation = 'softmax')
    ])
    
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values = [1e-2, 1e-3])),
                 loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
    return model
        
    

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [33]:
tuner_search = RandomSearch(build_model, objective = "val_accuracy", max_trials = 5, directory = "output",project_name = "Fashion_MNIST" )

INFO:tensorflow:Reloading Oracle from existing project output/Fashion_MNIST/oracle.json
INFO:tensorflow:Reloading Tuner from output/Fashion_MNIST/tuner0.json


In [34]:
tuner_search.search(train_images, train_labels, epochs = 3, validation_split = 0.1)

INFO:tensorflow:Oracle triggered exit


In [35]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 80)        69200     
_________________________________________________________________
flatten (Flatten)            (None, 38720)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                3717216   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 3,789,882
Trainable params: 3,789,882
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(train_images, train_labels, epochs = 10, validation_split = 0.1, initial_epoch = 3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1564 - accuracy: 0.9411 - val_loss: 0.2594 - val_accuracy: 0.9067
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1166 - accuracy: 0.9567 - val_loss: 0.2752 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0850 - accuracy: 0.9681 - val_loss: 0.3255 - val_accuracy: 0.9075
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0637 - accuracy: 0.9763 - val_loss: 0.3650 - val_accuracy: 0.9113
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0494 - accuracy: 0.9814 - val_loss: 0.4071 - val_accuracy: 0.9078
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0389 - accuracy: 0.9854 - val_loss: 0.4685 - val_accuracy: 0.9115
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0314 - accuracy: 0.9881 - val_loss: 0.4933 - val_accuracy